# GIFT-Riemann Phase 1 Validation

## Réponse au Conseil des IAs

Ce notebook implémente les 4 tests critiques recommandés par le conseil (Opus, Gemini, Grok, Kimi, GPT).

### Tests Implémentés

| Test | Question | Métrique |
|------|----------|----------|
| **1. Unfolded Error** | Prédit-on la structure fine ou juste la tendance? | Erreur en spacings |
| **2. Detrending** | Le signal survit-il après retrait de l'asymptotique? | Erreur sur résidus |
| **3. Train/Test Gelé** | Généralisation hors échantillon? | Erreur test vs train |
| **4. Sensibilité Lags** | Les lags Fibonacci sont-ils spéciaux? | Comparaison erreurs |

### Critère de Succès

- **Test 1** : Erreur unfolded moyenne < 1 spacing (idéalement < 0.5)
- **Test 2** : Signal persiste après détrendage (erreur < baseline)
- **Test 3** : Erreur test ≤ 1.5× erreur train
- **Test 4** : Lags [5,8,13,27] significativement meilleurs que voisins

In [ ]:
import numpy as np
import json
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print("Phase 1 Validation - Conseil des IAs")
print("="*50)

## Chargement des Données

Charger les zéros de Riemann (fichier ZEROS.txt ou génération).

In [ ]:
# Tentative de chargement du fichier ZEROS.txt
import os

def load_zeros(filepath: str = None) -> np.ndarray:
    """Charge les zéros depuis fichier ou utilise les premiers connus."""
    
    # Essayer plusieurs chemins possibles
    possible_paths = [
        filepath,
        'ZEROS.txt',
        '../ZEROS.txt',
        '/content/ZEROS.txt',
        'zeros.txt'
    ]
    
    for path in possible_paths:
        if path and os.path.exists(path):
            print(f"Chargement depuis {path}...")
            zeros = []
            with open(path, 'r') as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#'):
                        try:
                            zeros.append(float(line))
                        except ValueError:
                            continue
            if zeros:
                print(f"Chargé {len(zeros)} zéros")
                return np.array(zeros)
    
    # Fallback: premiers 1000 zéros (approximation haute précision)
    print("Fichier non trouvé - utilisation des zéros intégrés")
    print("Pour des résultats complets, uploadez ZEROS.txt")
    
    # Premiers 100 zéros avec haute précision (Odlyzko)
    first_100 = [
        14.134725141734693, 21.022039638771554, 25.010857580145688,
        30.424876125859513, 32.935061587739189, 37.586178158825671,
        40.918719012147495, 43.327073280914999, 48.005150881167159,
        49.773832477672302, 52.970321477714460, 56.446247697063394,
        59.347044002602353, 60.831778524609809, 65.112544048081606,
        67.079810529494173, 69.546401711173979, 72.067157674481907,
        75.704690699083933, 77.144840068874805, 79.337375020249367,
        82.910380854086030, 84.735492980517050, 87.425274613125229,
        88.809111207634465, 92.491899270558484, 94.651344040519848,
        95.870634228245309, 98.831194218193692, 101.31785100573139,
        103.72553804047833, 105.44662305232609, 107.16861118427640,
        111.02953554316967, 111.87465917699263, 114.32022091545271,
        116.22668032085755, 118.79078286597621, 121.37012500242064,
        122.94682929355258, 124.25681855434576, 127.51668387959649,
        129.57870419995605, 131.08768853093265, 133.49773720299758,
        134.75650975337387, 138.11604205453344, 139.73620895212138,
        141.12370740402112, 143.11184580762063, 146.00098248149497,
        147.42276534331817, 150.05352042078194, 150.92525769811311,
        153.02469388971455, 156.11290929488189, 157.59759166468790,
        158.84998819298678, 161.18896413581623, 163.03070933026669,
        165.53706943428540, 167.18443987337141, 169.09451541594776,
        169.91197647941924, 173.41153673461777, 174.75419152717550,
        176.44143402671451, 178.37740777581620, 179.91648402025142,
        182.20707848436646, 184.87446784737076, 185.59878367569748,
        187.22892258423594, 189.41615865188581, 192.02665636225166,
        193.07972660984527, 195.26539667784402, 196.87648178679182,
        198.01530951432770, 201.26475194370426, 202.49359427372137,
        204.18967180042432, 205.39469720895602, 207.90625898483556,
        209.57650984378520, 211.69086259334878, 213.34791926879517,
        214.54704478344582, 216.16953848996527, 219.06759635319410,
        220.71491881384926, 221.43070552767637, 224.00700025498247,
        224.98325235953609, 227.42144502665364, 229.33741330917844,
        231.25018870093929, 231.98715902637730, 233.69340417045408,
        236.52422966581694
    ]
    return np.array(first_100)

gamma = load_zeros()
N_zeros = len(gamma)
print(f"\nPlage: γ₁ = {gamma[0]:.4f} à γ_{N_zeros} = {gamma[-1]:.4f}")

## Fonctions Utilitaires

In [ ]:
def local_spacing(T: float) -> float:
    """
    Espacement moyen local entre zéros à hauteur T.
    Δ(T) ≈ 2π / log(T / 2π)
    
    Ref: Montgomery pair correlation, Odlyzko
    """
    if T <= 2 * np.pi:
        return 1.0  # Fallback pour petits T
    return 2 * np.pi / np.log(T / (2 * np.pi))


def riemann_asymptotic(n: int) -> float:
    """
    Approximation asymptotique de γₙ via Riemann-von Mangoldt.
    
    N(T) ≈ (T/2π) log(T/2π) - T/2π + 7/8
    
    Inversion: γₙ ≈ 2πn / log(n) pour grands n
    Correction: γₙ ≈ 2πn / W(n/e) où W = Lambert W
    """
    if n <= 0:
        return 0.0
    
    # Approximation de Lambert W via itération
    # W(x) ≈ log(x) - log(log(x)) pour grand x
    x = n / np.e
    if x < 1:
        return 2 * np.pi * n / max(np.log(n + 1), 1)
    
    w = np.log(x)
    if w > 1:
        w = w - np.log(w)
    
    # Raffinement Newton (quelques itérations)
    for _ in range(5):
        ew = np.exp(w)
        w = w - (w * ew - x) / (ew * (w + 1))
    
    return 2 * np.pi * n / w


def fit_recurrence(gamma: np.ndarray, lags: List[int], 
                   start: int = None, end: int = None) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Fit une récurrence linéaire γₙ = Σᵢ aᵢ γₙ₋ₗᵢ + c
    
    Retourne: (coefficients, predictions, errors)
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    # Construction matrice
    X = []
    y = []
    indices = []
    
    for n in range(start, end):
        row = [gamma[n - lag] for lag in lags] + [1.0]  # +1 pour constante
        X.append(row)
        y.append(gamma[n])
        indices.append(n)
    
    X = np.array(X)
    y = np.array(y)
    
    # Least squares
    coeffs, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    # Predictions
    y_pred = X @ coeffs
    
    return coeffs, y_pred, y, np.array(indices)


# Test rapide
print(f"Spacing à γ₁₀₀ ≈ {gamma[99]:.1f}: Δ = {local_spacing(gamma[99]):.4f}")
print(f"Asymptotique γ₁₀₀ théorique: {riemann_asymptotic(100):.2f}")
print(f"Valeur réelle γ₁₀₀: {gamma[99]:.2f}")

---

# TEST 1 : Erreur Unfolded (en Spacings)

**Question** : Prédit-on la structure fine ou juste la tendance lisse ?

**Métrique** : 
$$e_n = \frac{\hat{\gamma}_n - \gamma_n}{\Delta(\gamma_n)}$$

où $\Delta(T) \approx \frac{2\pi}{\log(T/2\pi)}$ est l'espacement moyen local.

**Critère de succès** : |eₙ| < 1 en moyenne (idéalement < 0.5)

In [ ]:
def test_unfolded_error(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27]):
    """
    TEST 1: Calcule l'erreur en unités d'espacement (unfolded).
    """
    print("\n" + "="*60)
    print("TEST 1 : ERREUR UNFOLDED (en spacings)")
    print("="*60)
    
    max_lag = max(lags)
    if len(gamma) <= max_lag + 10:
        print(f"⚠️  Pas assez de données (n={len(gamma)}, besoin > {max_lag + 10})")
        return None
    
    # Fit sur toutes les données disponibles
    coeffs, y_pred, y_true, indices = fit_recurrence(gamma, lags)
    
    print(f"\nCoefficients fittés (lags {lags}):")
    for i, lag in enumerate(lags):
        print(f"  a_{lag} = {coeffs[i]:.6f}")
    print(f"  c = {coeffs[-1]:.6f}")
    
    # Calcul erreurs
    errors_abs = y_pred - y_true
    errors_rel_pct = np.abs(errors_abs) / y_true * 100
    
    # Erreurs unfolded (en spacings)
    spacings = np.array([local_spacing(g) for g in y_true])
    errors_unfolded = np.abs(errors_abs) / spacings
    
    print(f"\n📊 Statistiques sur n={len(y_true)} points:")
    print(f"\n  Erreur relative (%) - ancienne métrique:")
    print(f"    Moyenne: {np.mean(errors_rel_pct):.4f}%")
    print(f"    Médiane: {np.median(errors_rel_pct):.4f}%")
    
    print(f"\n  Erreur UNFOLDED (spacings) - nouvelle métrique:")
    print(f"    Moyenne: {np.mean(errors_unfolded):.4f} spacings")
    print(f"    Médiane: {np.median(errors_unfolded):.4f} spacings")
    print(f"    Max: {np.max(errors_unfolded):.4f} spacings")
    print(f"    Std: {np.std(errors_unfolded):.4f} spacings")
    
    # Distribution par quantiles
    print(f"\n  Distribution erreur unfolded:")
    for q in [50, 75, 90, 95, 99]:
        val = np.percentile(errors_unfolded, q)
        print(f"    {q}th percentile: {val:.4f} spacings")
    
    # Verdict
    mean_unfolded = np.mean(errors_unfolded)
    print(f"\n🎯 VERDICT TEST 1:")
    if mean_unfolded < 0.5:
        print(f"   ✅ PASS (erreur moyenne = {mean_unfolded:.3f} < 0.5 spacings)")
        print(f"   → On prédit la STRUCTURE FINE des zéros!")
        verdict = "PASS"
    elif mean_unfolded < 1.0:
        print(f"   ⚠️  MARGINAL (erreur moyenne = {mean_unfolded:.3f} < 1.0 spacing)")
        print(f"   → Prédiction à ~1 zéro près en moyenne")
        verdict = "MARGINAL"
    else:
        print(f"   ❌ FAIL (erreur moyenne = {mean_unfolded:.3f} > 1.0 spacing)")
        print(f"   → On prédit surtout la TENDANCE, pas la structure fine")
        verdict = "FAIL"
    
    return {
        'verdict': verdict,
        'mean_unfolded': float(mean_unfolded),
        'median_unfolded': float(np.median(errors_unfolded)),
        'mean_pct': float(np.mean(errors_rel_pct)),
        'coefficients': coeffs.tolist(),
        'n_points': len(y_true)
    }

# Exécution Test 1
test1_results = test_unfolded_error(gamma, lags=[5, 8, 13, 27])

---

# TEST 2 : Détrendage (Signal vs Tendance)

**Question** : Le signal survit-il après retrait de la tendance asymptotique ?

**Méthode** : 
1. Calculer $g(n) = $ approximation asymptotique de $\gamma_n$
2. Définir résidus $r_n = \gamma_n - g(n)$
3. Fitter la récurrence sur les résidus
4. Comparer à une baseline (AR simple ou moyenne mobile)

**Critère** : Si l'erreur sur résidus est significativement meilleure que baseline, le signal est réel.

In [ ]:
def test_detrending(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27]):
    """
    TEST 2: Test si le signal persiste après détrendage.
    """
    print("\n" + "="*60)
    print("TEST 2 : DÉTRENDAGE (Signal vs Tendance)")
    print("="*60)
    
    n_zeros = len(gamma)
    max_lag = max(lags)
    
    if n_zeros <= max_lag + 10:
        print(f"⚠️  Pas assez de données")
        return None
    
    # Calcul de la tendance asymptotique
    trend = np.array([riemann_asymptotic(n+1) for n in range(n_zeros)])
    residuals = gamma - trend
    
    print(f"\n📈 Tendance asymptotique:")
    print(f"   Erreur tendance moyenne: {np.mean(np.abs(residuals)):.4f}")
    print(f"   Erreur tendance relative: {np.mean(np.abs(residuals)/gamma)*100:.4f}%")
    
    # Fit récurrence sur résidus
    print(f"\n🔄 Fit récurrence sur RÉSIDUS (rₙ = γₙ - g(n)):")
    
    coeffs_res, y_pred_res, y_true_res, indices = fit_recurrence(residuals, lags)
    errors_res = np.abs(y_pred_res - y_true_res)
    
    print(f"   Coefficients sur résidus:")
    for i, lag in enumerate(lags):
        print(f"     a_{lag} = {coeffs_res[i]:.6f}")
    print(f"     c = {coeffs_res[-1]:.6f}")
    
    # Baseline 1: AR(1) simple sur résidus
    print(f"\n📊 Comparaison avec baselines:")
    
    # AR(1): rₙ ≈ α·rₙ₋₁ + c
    coeffs_ar1, y_pred_ar1, y_true_ar1, _ = fit_recurrence(residuals, [1])
    errors_ar1 = np.abs(y_pred_ar1 - y_true_ar1)
    
    # Moyenne mobile: rₙ ≈ moyenne des k derniers
    k = 5
    y_pred_ma = []
    y_true_ma = []
    for n in range(k, n_zeros):
        y_pred_ma.append(np.mean(residuals[n-k:n]))
        y_true_ma.append(residuals[n])
    errors_ma = np.abs(np.array(y_pred_ma) - np.array(y_true_ma))
    
    # Stats comparatives
    mean_gift = np.mean(errors_res)
    mean_ar1 = np.mean(errors_ar1)
    mean_ma = np.mean(errors_ma)
    
    print(f"\n   Erreur moyenne absolue sur résidus:")
    print(f"     GIFT [5,8,13,27]: {mean_gift:.6f}")
    print(f"     AR(1) baseline:   {mean_ar1:.6f}")
    print(f"     MA(5) baseline:   {mean_ma:.6f}")
    
    # Amélioration vs baselines
    improv_ar1 = (mean_ar1 - mean_gift) / mean_ar1 * 100
    improv_ma = (mean_ma - mean_gift) / mean_ma * 100
    
    print(f"\n   Amélioration GIFT vs baselines:")
    print(f"     vs AR(1): {improv_ar1:+.2f}%")
    print(f"     vs MA(5): {improv_ma:+.2f}%")
    
    # Verdict
    print(f"\n🎯 VERDICT TEST 2:")
    if improv_ar1 > 20 and improv_ma > 20:
        print(f"   ✅ PASS - Signal significatif après détrendage")
        print(f"   → La récurrence capture plus que la tendance lisse!")
        verdict = "PASS"
    elif improv_ar1 > 10 or improv_ma > 10:
        print(f"   ⚠️  MARGINAL - Légère amélioration vs baselines")
        verdict = "MARGINAL"
    else:
        print(f"   ❌ FAIL - Pas d'amélioration significative")
        print(f"   → La récurrence capture principalement la tendance")
        verdict = "FAIL"
    
    return {
        'verdict': verdict,
        'mean_error_gift': float(mean_gift),
        'mean_error_ar1': float(mean_ar1),
        'mean_error_ma': float(mean_ma),
        'improvement_vs_ar1': float(improv_ar1),
        'improvement_vs_ma': float(improv_ma)
    }

# Exécution Test 2
test2_results = test_detrending(gamma, lags=[5, 8, 13, 27])

---

# TEST 3 : Train/Test Gelé (Généralisation)

**Question** : Les coefficients généralisent-ils hors échantillon ?

**Méthode** :
1. Fitter sur [max_lag, N/2] (TRAIN)
2. **Geler** les coefficients
3. Prédire sur [N/2+1, N] (TEST)
4. Comparer les erreurs

**Critère** : Erreur TEST ≤ 1.5× erreur TRAIN

In [ ]:
def test_train_test_split(gamma: np.ndarray, lags: List[int] = [5, 8, 13, 27]):
    """
    TEST 3: Validation train/test avec coefficients gelés.
    """
    print("\n" + "="*60)
    print("TEST 3 : TRAIN/TEST GELÉ (Généralisation)")
    print("="*60)
    
    n_zeros = len(gamma)
    max_lag = max(lags)
    
    if n_zeros <= max_lag + 20:
        print(f"⚠️  Pas assez de données pour split")
        return None
    
    # Split 50/50
    split_idx = n_zeros // 2
    
    print(f"\n📊 Split des données:")
    print(f"   TRAIN: n = {max_lag+1} à {split_idx} ({split_idx - max_lag} points)")
    print(f"   TEST:  n = {split_idx+1} à {n_zeros} ({n_zeros - split_idx} points)")
    
    # Fit sur TRAIN uniquement
    coeffs_train, y_pred_train, y_true_train, _ = fit_recurrence(
        gamma, lags, start=max_lag, end=split_idx
    )
    
    print(f"\n🔧 Coefficients (fit sur TRAIN seulement):")
    for i, lag in enumerate(lags):
        print(f"   a_{lag} = {coeffs_train[i]:.6f}")
    print(f"   c = {coeffs_train[-1]:.6f}")
    
    # Appliquer coefficients gelés sur TEST
    y_pred_test = []
    y_true_test = []
    
    for n in range(split_idx, n_zeros):
        pred = sum(coeffs_train[i] * gamma[n - lag] for i, lag in enumerate(lags))
        pred += coeffs_train[-1]  # constante
        y_pred_test.append(pred)
        y_true_test.append(gamma[n])
    
    y_pred_test = np.array(y_pred_test)
    y_true_test = np.array(y_true_test)
    
    # Erreurs relatives
    errors_train_pct = np.abs(y_pred_train - y_true_train) / y_true_train * 100
    errors_test_pct = np.abs(y_pred_test - y_true_test) / y_true_test * 100
    
    # Erreurs unfolded
    spacings_train = np.array([local_spacing(g) for g in y_true_train])
    spacings_test = np.array([local_spacing(g) for g in y_true_test])
    
    errors_train_unf = np.abs(y_pred_train - y_true_train) / spacings_train
    errors_test_unf = np.abs(y_pred_test - y_true_test) / spacings_test
    
    print(f"\n📈 Résultats:")
    print(f"\n   Erreur relative (%):")
    print(f"     TRAIN: {np.mean(errors_train_pct):.4f}%")
    print(f"     TEST:  {np.mean(errors_test_pct):.4f}%")
    print(f"     Ratio TEST/TRAIN: {np.mean(errors_test_pct)/np.mean(errors_train_pct):.2f}x")
    
    print(f"\n   Erreur unfolded (spacings):")
    print(f"     TRAIN: {np.mean(errors_train_unf):.4f}")
    print(f"     TEST:  {np.mean(errors_test_unf):.4f}")
    ratio_unf = np.mean(errors_test_unf) / np.mean(errors_train_unf)
    print(f"     Ratio TEST/TRAIN: {ratio_unf:.2f}x")
    
    # Verdict
    print(f"\n🎯 VERDICT TEST 3:")
    if ratio_unf <= 1.5:
        print(f"   ✅ PASS (ratio = {ratio_unf:.2f} ≤ 1.5)")
        print(f"   → Bonne généralisation hors échantillon!")
        verdict = "PASS"
    elif ratio_unf <= 2.0:
        print(f"   ⚠️  MARGINAL (ratio = {ratio_unf:.2f} ≤ 2.0)")
        verdict = "MARGINAL"
    else:
        print(f"   ❌ FAIL (ratio = {ratio_unf:.2f} > 2.0)")
        print(f"   → Overfitting probable")
        verdict = "FAIL"
    
    return {
        'verdict': verdict,
        'train_error_pct': float(np.mean(errors_train_pct)),
        'test_error_pct': float(np.mean(errors_test_pct)),
        'train_error_unf': float(np.mean(errors_train_unf)),
        'test_error_unf': float(np.mean(errors_test_unf)),
        'ratio': float(ratio_unf),
        'coefficients_frozen': coeffs_train.tolist()
    }

# Exécution Test 3
test3_results = test_train_test_split(gamma, lags=[5, 8, 13, 27])

---

# TEST 4 : Sensibilité aux Lags (Structure Fibonacci)

**Question** : Les lags [5,8,13,27] sont-ils vraiment spéciaux ?

**Méthode** :
1. Tester des lags voisins (±1 sur chaque composante)
2. Tester des lags arithmétiques et aléatoires
3. Comparer les erreurs

**Critère** : Lags Fibonacci doivent être significativement meilleurs

In [ ]:
def test_lag_sensitivity(gamma: np.ndarray, reference_lags: List[int] = [5, 8, 13, 27]):
    """
    TEST 4: Test de sensibilité aux lags.
    """
    print("\n" + "="*60)
    print("TEST 4 : SENSIBILITÉ AUX LAGS")
    print("="*60)
    
    n_zeros = len(gamma)
    max_lag = max(reference_lags)
    
    if n_zeros <= max_lag + 10:
        print(f"⚠️  Pas assez de données")
        return None
    
    def get_error(lags: List[int]) -> float:
        """Retourne l'erreur unfolded moyenne pour des lags donnés."""
        try:
            max_l = max(lags)
            if max_l >= n_zeros - 5:
                return float('inf')
            coeffs, y_pred, y_true, _ = fit_recurrence(gamma, lags)
            spacings = np.array([local_spacing(g) for g in y_true])
            errors = np.abs(y_pred - y_true) / spacings
            return np.mean(errors)
        except:
            return float('inf')
    
    # Erreur de référence (Fibonacci)
    error_ref = get_error(reference_lags)
    print(f"\n📊 Référence GIFT [5,8,13,27]: {error_ref:.6f} spacings")
    print(f"   Propriétés: 5+8=13 ✓, 5×8-13=27 ✓")
    
    # Test 4a: Lags voisins (perturbation ±1)
    print(f"\n🔍 Test 4a: Lags voisins (perturbation ±1)")
    neighbor_tests = [
        [4, 8, 13, 27],  # -1 sur 5
        [6, 8, 13, 27],  # +1 sur 5
        [5, 7, 13, 27],  # -1 sur 8
        [5, 9, 13, 27],  # +1 sur 8
        [5, 8, 12, 27],  # -1 sur 13
        [5, 8, 14, 27],  # +1 sur 13 (14 = dim(G₂)!)
        [5, 8, 13, 26],  # -1 sur 27
        [5, 8, 13, 28],  # +1 sur 27
    ]
    
    neighbor_errors = []
    for lags in neighbor_tests:
        err = get_error(lags)
        neighbor_errors.append(err)
        status = "✓" if err > error_ref else "✗"
        fib_check = "" 
        if lags[0] + lags[1] == lags[2]:
            fib_check += f" ({lags[0]}+{lags[1]}={lags[2]})"
        print(f"   {lags}: {err:.6f} {status}{fib_check}")
    
    # Test 4b: Lags alternatifs structurés
    print(f"\n🔍 Test 4b: Lags alternatifs structurés")
    alt_tests = [
        ([3, 5, 8, 13], "Fibonacci pur"),
        ([5, 8, 13, 21], "Fibonacci étendu (21=b₂)"),
        ([7, 14, 21, 28], "Multiples de 7 (dim K₇)"),
        ([8, 14, 21, 27], "GIFT: rank(E₈), dim(G₂), b₂, J₃𝕆"),
        ([5, 10, 15, 20], "Arithmétique (pas=5)"),
        ([4, 8, 12, 16], "Puissances de 2 ×"),
    ]
    
    alt_errors = []
    for lags, desc in alt_tests:
        if max(lags) < n_zeros - 5:
            err = get_error(lags)
            alt_errors.append((lags, err, desc))
            status = "✓" if err > error_ref else "★"
            print(f"   {lags} ({desc}): {err:.6f} {status}")
    
    # Test 4c: Lags aléatoires (baseline)
    print(f"\n🔍 Test 4c: 20 sets de lags aléatoires")
    np.random.seed(42)
    random_errors = []
    for i in range(20):
        rand_lags = sorted(np.random.choice(range(3, min(30, n_zeros-5)), 4, replace=False).tolist())
        err = get_error(rand_lags)
        random_errors.append(err)
    
    print(f"   Erreur moyenne random: {np.mean(random_errors):.6f}")
    print(f"   Erreur min random: {np.min(random_errors):.6f}")
    print(f"   Erreur max random: {np.max(random_errors):.6f}")
    
    # Analyse
    print(f"\n📈 Analyse:")
    
    n_neighbors_worse = sum(1 for e in neighbor_errors if e > error_ref)
    print(f"   Voisins pires que [5,8,13,27]: {n_neighbors_worse}/{len(neighbor_errors)}")
    
    percentile = sum(1 for e in random_errors if e > error_ref) / len(random_errors) * 100
    print(f"   Percentile vs random: {percentile:.1f}%")
    
    # Trouver si un autre set est meilleur
    all_tested = [(reference_lags, error_ref, "GIFT Fibonacci")]
    all_tested.extend([(neighbor_tests[i], neighbor_errors[i], f"Voisin #{i+1}") 
                       for i in range(len(neighbor_tests))])
    all_tested.extend(alt_errors)
    
    best = min(all_tested, key=lambda x: x[1])
    print(f"\n   🏆 Meilleur set trouvé: {best[0]} ({best[2]})")
    print(f"      Erreur: {best[1]:.6f}")
    
    # Verdict
    print(f"\n🎯 VERDICT TEST 4:")
    
    is_best = (best[0] == reference_lags)
    neighbors_stable = n_neighbors_worse >= 6  # Au moins 6/8 voisins pires
    beats_random = percentile >= 80
    
    if is_best and neighbors_stable:
        print(f"   ✅ PASS - [5,8,13,27] est optimal et stable")
        print(f"   → Structure Fibonacci confirmée!")
        verdict = "PASS"
    elif neighbors_stable or beats_random:
        print(f"   ⚠️  MARGINAL - Lags bons mais pas uniques")
        if not is_best:
            print(f"   → {best[0]} est légèrement meilleur")
        verdict = "MARGINAL"
    else:
        print(f"   ❌ FAIL - Lags [5,8,13,27] pas spéciaux")
        verdict = "FAIL"
    
    return {
        'verdict': verdict,
        'reference_error': float(error_ref),
        'neighbors_worse_count': n_neighbors_worse,
        'percentile_vs_random': float(percentile),
        'best_lags': best[0],
        'best_error': float(best[1]),
        'best_description': best[2]
    }

# Exécution Test 4
test4_results = test_lag_sensitivity(gamma, reference_lags=[5, 8, 13, 27])

---

# Synthèse des 4 Tests

In [ ]:
def final_synthesis(test1, test2, test3, test4):
    """
    Synthèse finale des 4 tests.
    """
    print("\n" + "="*60)
    print("SYNTHÈSE PHASE 1 - VALIDATION CONSEIL IAs")
    print("="*60)
    
    tests = [
        ("Test 1: Erreur Unfolded", test1),
        ("Test 2: Détrendage", test2),
        ("Test 3: Train/Test Gelé", test3),
        ("Test 4: Sensibilité Lags", test4),
    ]
    
    results = []
    print("\n📋 Résultats:")
    print()
    
    for name, result in tests:
        if result is None:
            verdict = "SKIP"
            icon = "⏭️"
        else:
            verdict = result.get('verdict', 'UNKNOWN')
            icon = {"PASS": "✅", "MARGINAL": "⚠️", "FAIL": "❌"}.get(verdict, "❓")
        
        results.append(verdict)
        print(f"   {icon} {name}: {verdict}")
    
    # Comptage
    n_pass = results.count("PASS")
    n_marginal = results.count("MARGINAL")
    n_fail = results.count("FAIL")
    n_skip = results.count("SKIP")
    
    print(f"\n📊 Score: {n_pass} PASS, {n_marginal} MARGINAL, {n_fail} FAIL, {n_skip} SKIP")
    
    # Verdict global
    print(f"\n" + "="*60)
    print("VERDICT GLOBAL")
    print("="*60)
    
    if n_pass >= 3:
        print("\n🏆 VALIDATION FORTE")
        print("   La récurrence GIFT-Riemann passe les tests critiques.")
        print("   → Prêt pour Phase 2 (théorie)")
        global_verdict = "STRONG"
    elif n_pass >= 2 and n_fail == 0:
        print("\n✅ VALIDATION MODÉRÉE")
        print("   Signal réel détecté, mais avec réserves.")
        print("   → Investigation supplémentaire recommandée")
        global_verdict = "MODERATE"
    elif n_pass >= 1 and n_fail <= 1:
        print("\n⚠️  VALIDATION FAIBLE")
        print("   Résultats mitigés - prudence requise.")
        global_verdict = "WEAK"
    else:
        print("\n❌ VALIDATION ÉCHOUÉE")
        print("   La récurrence ne passe pas les tests critiques.")
        print("   → Probablement un artefact de fitting")
        global_verdict = "FAILED"
    
    # Recommandations
    print(f"\n📝 Recommandations:")
    
    if test1 and test1.get('verdict') != 'PASS':
        print("   • Test 1: Tester sur plus de données (LMFDB 10M+ zéros)")
    
    if test2 and test2.get('verdict') != 'PASS':
        print("   • Test 2: Explorer coefficients fonction de n: aᵢ(n) = aᵢ⁰/log(n)")
    
    if test3 and test3.get('verdict') != 'PASS':
        print("   • Test 3: Étendre le test set, utiliser cross-validation")
    
    if test4 and test4.get('verdict') != 'PASS':
        print("   • Test 4: Explorer récurrence à 5 termes (ajouter lag 14 ou 21)")
    
    # Export résultats
    summary = {
        'global_verdict': global_verdict,
        'scores': {'pass': n_pass, 'marginal': n_marginal, 'fail': n_fail, 'skip': n_skip},
        'test1': test1,
        'test2': test2,
        'test3': test3,
        'test4': test4,
        'n_zeros': int(len(gamma)),
        'lags': [5, 8, 13, 27]
    }
    
    return summary

# Synthèse finale
summary = final_synthesis(test1_results, test2_results, test3_results, test4_results)

In [ ]:
# Export JSON pour analyse ultérieure
import json

def convert_to_serializable(obj):
    """Convertit les types numpy en types Python natifs."""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(v) for v in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    else:
        return obj

summary_clean = convert_to_serializable(summary)

print("\n" + "="*60)
print("EXPORT JSON")
print("="*60)
print(json.dumps(summary_clean, indent=2))

---

# Notes pour Tests avec Données Complètes

## Comment obtenir plus de zéros

1. **LMFDB** : https://www.lmfdb.org/zeros/zeta/ (jusqu'à 10⁹ zéros)
2. **Odlyzko** : Tables haute précision disponibles
3. **mpmath** : `from mpmath import zetazero; zetazero(n)` (lent mais précis)

## Prochaines étapes si validation passe

1. **Phase 2a** : Coefficients fonction de n (stabilisation)
2. **Phase 2b** : Récurrence 5-termes avec lag 14 ou 21
3. **Phase 3** : Connexion trace formula (Weil explicit formula)
4. **Phase 4** : PINN opérateur spectral